# Credit Card Fraud Detection

In [2]:
#importing necessary dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
#load dataset
data=pd.read_csv('creditcard.csv')
data.head(2)


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0


In [4]:
#information about data
data.info()
data['Class'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

0    284315
1       492
Name: Class, dtype: int64

In [5]:
#since 0 represents a normal transaction and 1 represents a fraudulent transcation, classifying the data for analysis 
valid=data[data.Class==0]
fraud=data[data.Class==1]
print(valid.shape,fraud.shape)


(284315, 31) (492, 31)


In [6]:
valid.Amount.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [7]:
fraud.Amount.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

Taking a sample dataset containing similar distribution of normal/valid transactions and Fraudulent transaction

In [8]:
valid_sample=valid.sample(n=492)

In [9]:
#concatenating two dataframes
final_data=pd.concat([valid_sample,fraud], axis=0)
final_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
256591,157767.0,-0.528298,-0.731876,0.017854,-3.491080,0.853715,-0.596612,0.098128,-0.022334,-0.237322,...,0.007164,0.487273,-0.522417,-1.354784,0.558119,0.162606,0.095420,0.102829,15.17,0
181858,125146.0,-1.634320,-0.447129,1.315312,-2.143882,0.837873,-0.692988,0.056571,0.124489,-1.271957,...,-0.828984,-2.004739,-0.094761,0.597199,0.167416,-0.530147,0.251773,0.069673,58.18,0
170245,120089.0,2.294695,-0.570808,-2.527375,-1.014332,0.172818,-1.496824,0.261161,-0.451515,-1.015845,...,0.481440,1.383841,-0.250283,-0.282524,0.719397,0.307609,-0.109606,-0.112056,1.79,0
129658,79173.0,1.131031,0.034660,0.738269,0.988709,-0.795204,-0.997443,0.076801,-0.215621,0.330045,...,-0.169801,-0.352971,0.041915,0.968439,0.391506,0.230748,-0.023755,0.025324,39.84,0
112047,72488.0,-0.705757,0.607085,1.372398,-1.212049,-0.012402,-0.318445,0.460012,0.141117,0.320315,...,-0.013709,0.155434,-0.236904,0.080626,-0.181921,0.891296,0.144108,0.161836,5.00,0


In [10]:
final_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,96549.912602,0.008334,-0.026282,0.022700,-0.091491,-0.089624,0.005897,-0.012958,-0.132810,0.009070,...,0.004683,-0.014175,0.040652,-0.014521,-0.027961,0.011726,0.002077,0.006015,0.016164,92.389289
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


Splitting the dataset into features and targets 

In [11]:
X=final_data.drop(columns='Class',axis=1)
Y=final_data['Class']

In [12]:
print(X)

            Time        V1        V2        V3        V4        V5        V6  \
256591  157767.0 -0.528298 -0.731876  0.017854 -3.491080  0.853715 -0.596612   
181858  125146.0 -1.634320 -0.447129  1.315312 -2.143882  0.837873 -0.692988   
170245  120089.0  2.294695 -0.570808 -2.527375 -1.014332  0.172818 -1.496824   
129658   79173.0  1.131031  0.034660  0.738269  0.988709 -0.795204 -0.997443   
112047   72488.0 -0.705757  0.607085  1.372398 -1.212049 -0.012402 -0.318445   
...          ...       ...       ...       ...       ...       ...       ...   
279863  169142.0 -1.927883  1.125653 -4.518331  1.749293 -1.566487 -2.010494   
280143  169347.0  1.378559  1.289381 -5.004247  1.411850  0.442581 -1.326536   
280149  169351.0 -0.676143  1.126366 -2.213700  0.468308 -1.120541 -0.003346   
281144  169966.0 -3.113832  0.585864 -5.399730  1.817092 -0.840618 -2.943548   
281674  170348.0  1.991976  0.158476 -2.583441  0.408670  1.151147 -0.096695   

              V7        V8        V9  .

In [13]:
print(Y)

256591    0
181858    0
170245    0
129658    0
112047    0
         ..
279863    1
280143    1
280149    1
281144    1
281674    1
Name: Class, Length: 984, dtype: int64


Split the dataset into training and testing sets

In [16]:
X_train,X_test,Y_train,Y_test= train_test_split(X,Y, test_size=0.2,stratify=Y,random_state=2)

Create and train the logistic regression model

In [17]:
model=LogisticRegression()
model.fit(X_train,Y_train)

LogisticRegression()

Predict accuracy score 

In [ ]:
X_train_prediction=model.predict(X_train)
training_accuracy=accuracy_score(X_train_prediction,Y_train)

In [ ]:
print("The accuracy score on Training Data : ",training_accuracy)

In [ ]:
X_test_prediction=model.predict(X_test)
test_accuracy=accuracy_score(X_test_prediction,Y_test)

In [ ]:
print("The accuracy score on Test Data : ",test_accuracy)